In [3]:
# Imported Libraries

import numpy as np # linear algebra
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections


# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import StratifiedShuffleSplit


2025-03-12 11:02:17.506479: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-12 11:02:17.697992: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-12 11:02:18.427171: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-12 11:02:21.857743: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
data1 = pd.read_csv('../raw_data/creditcard.csv')
data1.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df = data1.copy()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [7]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [8]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [9]:
df.isnull().sum().max()

0

In [10]:
df["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [11]:
print('Not Fraud', round(df['Class'].value_counts()[0]/len(df) * 100,2))
print('Fraud', round(df['Class'].value_counts()[1]/len(df) * 100,2))

Not Fraud 99.83
Fraud 0.17


preprocessing

In [12]:
from sklearn.preprocessing import RobustScaler

In [13]:
# Split the data into training and testing sets:
from sklearn.model_selection import train_test_split

X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [14]:
from imblearn.over_sampling import SMOTE
#SMOTE balances imbalanced data by generating
# synthetic samples for the minority class, preventing underfitting and improving model
# performance on classification tasks.
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [35]:
from sklearn.linear_model import LogisticRegression


# Train the model
model = LogisticRegression(
    class_weight='balanced',  # Handle imbalance class_weight='balanced'    automatically compensates for class imbalance.

    max_iter=1000,
    random_state=42
)
model.fit(X_train_res, y_train_res)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.08      0.91      0.14        98

    accuracy                           0.98     56962
   macro avg       0.54      0.94      0.57     56962
weighted avg       1.00      0.98      0.99     56962

ROC-AUC Score: 0.9445413259563814
F1 Score: 0.139937106918239


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Train the model
model = XGBClassifier(
    scale_pos_weight=(len(y_train_res) - sum(y_train_res)) / sum(y_train_res),  # Handle imbalance
    max_depth=5,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)
model.fit(X_train_res, y_train_res)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))

print("F1 Score:", f1_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.38      0.88      0.53        98

    accuracy                           1.00     56962
   macro avg       0.69      0.94      0.76     56962
weighted avg       1.00      1.00      1.00     56962

ROC-AUC Score: 0.9375445028883809
F1 Score: 0.5308641975308642


In [38]:
import xgboost as xgb

model = xgb.XGBClassifier(
    scale_pos_weight=(len(y_train_res) - sum(y_train_res)) / sum(y_train_res),  # Handle imbalance
    max_depth=5,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)
model.fit(X_train_res, y_train_res)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.38      0.88      0.53        98

    accuracy                           1.00     56962
   macro avg       0.69      0.94      0.76     56962
weighted avg       1.00      1.00      1.00     56962

ROC-AUC Score: 0.9375445028883809
F1 Score: 0.5308641975308642


In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Train the model
model = RandomForestClassifier(
    class_weight='balanced',  # Handle imbalance
    n_estimators=100,
    max_depth=5,
    random_state=42
)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.36      0.89      0.51        98

    accuracy                           1.00     56962
   macro avg       0.68      0.94      0.76     56962
weighted avg       1.00      1.00      1.00     56962

ROC-AUC Score: 0.9425234429731374
F1 Score: 0.5132743362831859


In [19]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.metrics import classification_report, roc_auc_score

# # Train the model
# model = GradientBoostingClassifier(
#     learning_rate=0.1,
#     n_estimators=100,
#     max_depth=5, 
# print(classification_report(y_test, y_pred))
# print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))


In [20]:
# from sklearn.svm import OneClassSVM
# from sklearn.metrics import classification_report, roc_auc_score, f1_score

# # Обучаем модель One-Class SVM
# model = OneClassSVM(nu=0.05, kernel="rbf", gamma="auto")  # nu - доля выбросов
# model.fit(X_train)

# # Предсказываем аномалии (1 – нормальные, -1 – выбросы)
# y_pred = model.predict(X_test)
# y_pred = [1 if i == 1 else 0 for i in y_pred]  # Приводим к 0 и 1

# # Оценка модели
# print(classification_report(y_test, y_pred))
# print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))
# print("F1 Score:", f1_score(y_test, y_pred))

# to slow


In [21]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Train Isolation Forest
model = IsolationForest(contamination=0.05, random_state=42)
model.fit(X_train)

# Predict anomalies (-1 = outlier, 1 = normal)
y_pred = model.predict(X_test)
y_pred = [1 if i == 1 else 0 for i in y_pred]  # Convert -1 to 0 (outliers)

# Model evaluation
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.97      0.05      0.10     56864
           1       0.00      0.13      0.00        98

    accuracy                           0.05     56962
   macro avg       0.49      0.09      0.05     56962
weighted avg       0.97      0.05      0.10     56962

ROC-AUC Score: 0.09175562459086055
F1 Score: 0.0004807425623578573


In [39]:
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np

# Convert data to numpy format
X_train_np = X_train.values
y_train_np = y_train.values
X_test_np = X_test.values
y_test_np = y_test.values

# Train TabNet model
model = TabNetClassifier()
model.fit(X_train_np, y_train_np, max_epochs=50, patience=10)

# Predict fraud
y_pred = model.predict(X_test_np)

# Model evaluation
print("Classification Report:\n", classification_report(y_test_np, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test_np, y_pred))
print("F1 Score:", f1_score(y_test_np, y_pred))


epoch 0  | loss: 0.03339 |  0:00:20s
epoch 1  | loss: 0.00671 |  0:00:33s
epoch 2  | loss: 0.00517 |  0:00:45s
epoch 3  | loss: 0.00481 |  0:01:00s
epoch 4  | loss: 0.00442 |  0:01:12s
epoch 5  | loss: 0.00427 |  0:01:26s
epoch 6  | loss: 0.00408 |  0:01:40s
epoch 7  | loss: 0.00471 |  0:01:54s
epoch 8  | loss: 0.00444 |  0:02:07s
epoch 9  | loss: 0.00395 |  0:02:21s
epoch 10 | loss: 0.00385 |  0:02:35s
epoch 11 | loss: 0.0039  |  0:02:49s
epoch 12 | loss: 0.00372 |  0:03:03s
epoch 13 | loss: 0.00373 |  0:03:16s
epoch 14 | loss: 0.00426 |  0:03:30s
epoch 15 | loss: 0.00407 |  0:03:43s
epoch 16 | loss: 0.0039  |  0:03:57s
epoch 17 | loss: 0.00402 |  0:04:11s
epoch 18 | loss: 0.00387 |  0:04:25s
epoch 19 | loss: 0.00384 |  0:04:38s
epoch 20 | loss: 0.00392 |  0:04:53s
epoch 21 | loss: 0.00376 |  0:05:07s
epoch 22 | loss: 0.00381 |  0:05:23s
epoch 23 | loss: 0.00365 |  0:05:36s
epoch 24 | loss: 0.00395 |  0:05:50s
epoch 25 | loss: 0.00369 |  0:06:05s
epoch 26 | loss: 0.00353 |  0:06:18s
e

In [44]:
import pandas as pd
from sklearn.metrics import recall_score, f1_score

# Store results for trained models
results = []

# Function to evaluate trained models
def evaluate_trained_model(name, y_test, y_pred):
    results.append({
        "Model": name,
        "Recall": recall_score(y_test, y_pred),
        "F1-score": f1_score(y_test, y_pred)
    })

# Collect results from all trained models
evaluate_trained_model("Logistic Regression", y_test, y_pred_logreg)
evaluate_trained_model("XGBoost", y_test, y_pred_xgb)
evaluate_trained_model("Random Forest", y_test, y_pred_rf)
evaluate_trained_model("Isolation Forest", y_test, y_pred_if)
evaluate_trained_model("TabNet", y_test_np, y_pred_tabnet)

# Display results in a table
df_results = pd.DataFrame(results)
import ace_tools as tools
tools.display_dataframe_to_user(name="Model Comparison", dataframe=df_results)


NameError: name 'y_pred_logreg' is not defined